In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
import glob
from LumFunc_miniJPAS import LumFunc_hist

from three_filter import three_filter_method

from scipy.integrate import simpson

from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

qso_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
qso_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
qso_zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# Randomly sample sources corresponding to 200 deg2
idx = np.random.randint(0, 100000, 510 * 200)
qso_flx = qso_flx[:, idx]
qso_err = qso_err[:, idx]
qso_zspec = qso_zspec[idx]

Lya_fts = pd.read_csv('csv/Lya_fts.csv')
EW_qso = np.abs(Lya_fts.LyaEW)[idx] / qso_zspec

# Apply errors
np.random.seed(22)
qso_flx += qso_err * np.random.normal(size=qso_err.shape)

In [ ]:
## Load SF catalog

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-5/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

sf_flx = data.to_numpy()[:, 1 : 60 + 1].T
sf_err = data.to_numpy()[:, 60 + 1 : 120 + 1].T

mag_noerr = flux_to_mag(sf_flx, w_central.reshape(-1, 1))
mag_noerr[np.isnan(mag_noerr)] = 99.

sf_flx += np.random.normal(size=(sf_err.shape)) * sf_err

mag = flux_to_mag(sf_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.

files2 = []
files3 = []
for i in range(len(files)):
    files2.append(f'{filename}SEDs{i + 1}.csv')
    files2.sort()
    files3.append(f'{filename}SEDs_no_line{i + 1}.csv')
    files3.sort()
fi = []
for name in files2:
    fi.append(pd.read_csv(name, header=None))
fi3 = []
for name in files3:
    fi3.append(pd.read_csv(name, header=None))

mock = {}
mock['SEDs'] = pd.concat(fi, axis=0, ignore_index=True).to_numpy()
mock['SEDs_no_line'] = pd.concat(fi, axis=0, ignore_index=True).to_numpy()
mock['w_Arr'] = np.load(filename + 'w_Arr.npy')

EW_sf = data['EW0'].to_numpy()
sf_zspec = data['z'].to_numpy()

In [ ]:
pm_flx = np.hstack((qso_flx, sf_flx))
pm_err = np.hstack((qso_err, sf_err))
zspec = np.concatenate((qso_zspec, sf_zspec))
EW_lya = np.concatenate((EW_qso, EW_sf))

N_sf = sf_flx.shape[1]
N_qso = qso_flx.shape[1]

qso_dL = cosmo.luminosity_distance(qso_zspec).to(u.cm).value
qso_L = EW_qso * Lya_fts.LyaCont[idx] * 1e-17 * (4 * np.pi * qso_dL**2)

sf_dL = cosmo.luminosity_distance(sf_zspec).to(u.cm).value
for src in range(N_sf):
    interval = slice(
        np.argmin(np.abs((1 + sf_zspec[src]) * w_lya - 6 - mock['w_Arr'])),
        np.argmin(np.abs((1 + sf_zspec[src]) * w_lya + 6 - mock['w_Arr'])),
        1
    )
    sf_cont = simpson(mock['SEDs_no_line'][src][interval], mock['w_Arr'][interval])
sf_L = EW_sf * sf_cont * (4 * np.pi * sf_dL**2)

L_lya = np.log10(np.concatenate((qso_L, sf_L)))

del sf_flx, sf_err, qso_flx, qso_err, sf_zspec,  qso_zspec, EW_sf, EW_qso, sf_dL, qso_dL, sf_L, qso_L

In [ ]:
w_lya = 1215.67 # A
N_sources = pm_flx.shape[1]
N_sources

In [ ]:
mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, 30)
lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    160, obs=True)
other_lines = identify_lines(line_other, pm_flx, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.12

In [ ]:
mag_min = 18
mag_max = 30

z_min = 2.5
z_max = 3.5
z_cut = (z_min < z_Arr) & (z_Arr < z_max)

nice_lya, bl = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr, give_bad_lines=True
) #& (mag[-2] > mag_min) & z_cut & (mag[-2] < mag_max)
nice_lya = nice_lya & (mag[-2] > mag_min) & z_cut & (mag[-2] < mag_max)

In [ ]:
## Fractions QSO / SF

good_qso = len(np.where(np.where(nice_lya & nice_z)[0] < N_qso)[0])
bad_qso = len(np.where(np.where(nice_lya & ~nice_z)[0] < N_qso)[0])
N_sel = count_true(nice_lya)
N_sel_good = count_true(nice_lya & nice_z)


good_frac = good_qso / N_sel_good
bad_frac = bad_qso / (N_sel - N_sel_good)

purity = count_true(nice_z & nice_lya) / count_true(nice_lya)

print(f'Good QSOs: {good_qso} | Bad QSOs: {bad_qso}')
print(f'Good SFs: {N_sel_good - good_qso} | Bad SFs: {N_sel - N_sel_good - bad_qso}')
print()
print('Good frac: {0:0.2f}'.format(good_frac))
print('Bad frac: {0:0.2f}'.format(bad_frac))
print()
print('Purity = {0:0.2f}'.format(purity))

In [ ]:
def EW_err(fnb, fnb_err, fcont, fcont_err, z, z_err, fwhm):
    e1 = fnb_err * fwhm / fcont / (1 + z)
    e2 = fcont_err * fwhm / (-fcont ** -2 * (1 + z))
    e3 = z_err * fwhm * (fnb - fcont) / fcont * (-1) / ((1 + z) ** 2)

    return (e1**2 + e2**2 + e3**2) ** 0.5

In [ ]:
EW_nb_Arr = np.zeros(N_sources)
EW_nb_e = np.copy(EW_nb_Arr)
L_Arr = np.zeros(N_sources)
flambda_Arr = np.zeros(N_sources)

for src in range(N_sources):
    l = lya_lines[src]
    flambda = pm_flx[l, src] - cont_est_lya[l, src]
    EW_nb_Arr[src] = nb_fwhm_Arr[l] * flambda\
        / cont_est_lya[l, src] / (1 + np.array(z_Arr[src]))
    EW_nb_e[src] = EW_err(
        pm_flx[l, src], pm_err[l, src], cont_est_lya[l, src], cont_err_lya[l, src],
        z_Arr[src], 0.06, 147
    )

    dL = cosmo.luminosity_distance(z_Arr[src]).to(u.cm).value
    L_Arr[src] = np.log10(147 * (flambda)\
        * 4 * np.pi * dL ** 2)
    flambda_Arr[src] = flambda

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

h = ax.hist2d(zspec[nice_lya], z_Arr[nice_lya],
    bins=(np.linspace(0, 3.5, 15), np.linspace(2.5, 3.5, 15)))
fig.colorbar(h[3], ax=ax)

x = np.linspace(2, 4, 100)
ax.plot(x, x, c='red')

ax.set_xlabel('z$_\mathrm{spec}$', fontsize=15)
ax.set_ylabel('Retrieved z', fontsize=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(18, 26, 20)

ax.hist(mag[-2][nice_lya & z_cut & nice_z], label='Good z', histtype='step', bins=bins)
ax.hist(mag[-2][nice_lya & z_cut & ~nice_z], label='Bad z', histtype='step', bins=bins)

ax.legend(fontsize=15, loc=2)

plt.show()

####

fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(0, 300, 30)

goodh = EW_nb_Arr[nice_lya & nice_z]
badh = EW_nb_Arr[nice_lya & ~nice_z]

ax.hist(goodh, label='Good z', histtype='step', bins=bins)
ax.hist(badh, label='Bad z', histtype='step', bins=bins)

ax.legend(fontsize=15, loc=2)

plt.show()

####

fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(43, 46, 30)

goodh = L_Arr[nice_lya & z_cut & nice_z]
badh = L_Arr[nice_lya & z_cut & ~nice_z]

ax.hist(goodh, label='Good z', histtype='step', bins=bins)
ax.hist(badh, label='Bad z', histtype='step', bins=bins)

ax.legend(fontsize=15, loc=2)

plt.show()

####

fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(-18.4, -15, 30)

goodh = np.log10(flambda_Arr[nice_lya & z_cut & nice_z])
badh = np.log10(flambda_Arr[nice_lya & z_cut & ~nice_z])

ax.hist(goodh, label='Good z', histtype='step', bins=bins)
ax.hist(badh, label='Bad z', histtype='step', bins=bins)

ax.legend(fontsize=15, loc=2)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

is_qso = np.concatenate((np.ones(N_qso), np.zeros(N_sf))).astype(bool)

ax.scatter(L_lya[nice_lya & nice_z & is_qso], L_Arr[nice_lya & nice_z & is_qso],
    label='QSO', alpha=0.5)
ax.scatter(L_lya[nice_lya & nice_z & ~is_qso], L_Arr[nice_lya & nice_z & ~is_qso],
    label='SF', alpha=0.5)
x = np.linspace(40, 48, 100)
ax.plot(x, x, linestyle='--', color='red', label='1:1')

ax.set_ylabel('Retrieved $\log L$', fontsize=15)
ax.set_xlabel('Real $\log L$', fontsize=15)

ax.set_ylim((42, 47))
ax.set_xlim((42, 47))

ax.legend(fontsize=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(43, 45.5, 15 + 1)
bin_centers = [(bins[k] + bins[k + 1]) / 2 for k in range(len(bins) - 1)]

goodh = L_Arr[nice_lya & z_cut & nice_z]
badh = L_Arr[nice_lya & z_cut & ~nice_z]

hg, bg = np.histogram(goodh, bins=bins)
hb, _ = np.histogram(badh, bins=bins)

phistar1 = 3.33e-6
Lstar1 = 10 ** 44.65
alpha1 = -1.35
phistar2 = 10 ** -3.45
Lstar2 = 10 ** 42.93
alpha2 = -1.93

volume = z_volume(z_min, z_max, 200)

totals = []
for b, _ in enumerate(bin_centers):
    Lx = 10 ** np.linspace(bins[b], bins[b + 1], 10)

    totals.append(
        simpson( 
            double_schechter(
                Lx, phistar2, Lstar2, alpha2, phistar1, Lstar1, alpha1
            ),
            Lx
        )
        * volume
    )

totals = np.array(totals)

ax.step(bin_centers, hg / totals, label='Completeness')
ax.step(bin_centers, hg / (hg + hb), label='Purity')
# ax.step(bin_centers, hg / (hg + hb) / (hg / totals))

ax.set_xlabel('$\log L$', fontsize=15)

ax.set_xlim((43, 45.5))
ax.set_ylim((0, 1.1))
ax.legend(fontsize=15)

plt.show()

correct = hg / (hg + hb) / (hg / totals) 

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(-16, -13, 15 + 1)
bin_centers = [(bins[k] + bins[k + 1]) / 2 for k in range(len(bins) - 1)]

goodh = np.log10(flambda_Arr[nice_lya & z_cut & nice_z] * 147)
badh = np.log10(flambda_Arr[nice_lya & z_cut & ~nice_z] * 147)

hg, bg = np.histogram(goodh, bins=bins)
hb, _ = np.histogram(badh, bins=bins)

ax.step(bin_centers, hg / totals, label='Completeness')
ax.step(bin_centers, hg / (hg + hb), label='Purity')

ax.set_xlabel('$\log L$', fontsize=15)

ax.legend(fontsize=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

ax.step(bin_centers, totals / 150)
ax.set_yscale('log')

ax.set_ylabel('N', fontsize=15)
ax.set_xlabel('$\log L$', fontsize=15)
ax.set_title('Expected sources in 1$\,\mathrm{deg}^2$', fontsize=20)

plt.show()

In [ ]:
total_hist = np.zeros(15)
volume = 0

for nb_c in range(56):
    LAE_src = (np.array(lya_lines) == nb_c)
    cont = cont_est_lya[nb_c, nice_lya & LAE_src]
    fl = pm_flx[nb_c, nice_lya & LAE_src] - cont
    z_sel = (w_central[nb_c] / w_lya - 1) * np.ones(len(fl))


    LF_bins, LF_hist, volume_i, bin_width = LumFunc_hist(
        fl, w_central[nb_c], nb_fwhm_Arr[nb_c],
        n_bins=15, L_min=43, L_max=45.5
    )
    total_hist += LF_hist
    volume += volume_i
volume = z_volume(z_min, z_max, 200)

fig, ax = plt.subplots(figsize=(9, 7))

ax.errorbar(LF_bins, total_hist / volume / bin_width,
    yerr= total_hist ** 0.5 / volume / bin_width,
    marker='.', linestyle='', markersize=15, label='Uncorrected')
ax.errorbar(LF_bins, total_hist / volume / bin_width * correct,
    yerr= (total_hist * correct) ** 0.5 / volume / bin_width,
    marker='.', linestyle='', markersize=15, label='Corrected')

Lx = np.linspace(10 ** 43, 10 ** 45.5, 1000)
Phi = double_schechter(
                Lx, phistar2, Lstar2, alpha2, phistar1, Lstar1, alpha1
            ) * Lx * np.log(10)

plt.plot(np.log10(Lx), Phi)

ax.set_ylim(1e-8, 1e-4)
ax.set_xlim(42.5, 45.5)
ax.set_yscale('log')

ax.legend(fontsize=15)

plt.show()

In [ ]:
for c, src in enumerate(np.where(~nice_lya & (L_lya > 45) & (zspec > 2.5))[0]):
    if c == 20: break
    fig = plt.figure(figsize=(15, 6))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
    print(f'nice_lya = {nice_lya[src]}')
    print(f'nice_z = {nice_z[src]}')
    print(f'good_lines = {bl[src]}')
    print(f'r = {mag[-2][src]}')
    print(f'src{src}')
    print(f'z_NB = {z_Arr[src]}')
    print(f'zspec = {zspec[src]}')
    print('EW0 = {0:0.1f} +- {1:0.1f}'.format(EW_nb_Arr[src], EW_nb_e[src]))
    # print(f'EW0_lya = {EW_lya[src]}')
    ax.axvline(w_central[lya_lines[src]], label='Selected NB')
    if zspec[src] > 1.9:
        ax.axvline(w_lya * (1 + zspec[src]), ls='--', c='red', label=r'Lya $\lambda$')
    ax.plot(w_central[1:54], cont_est_lya[1:-2, src], ls='--', c='black')
    for l in other_lines[src]:
        ax.axvline(w_central[l], ls='--', c='dimgray')

    ax.legend(fontsize=13)
    plt.show()

In [ ]:
correct